# Read the data

In [1]:
# read the data
data = sc.textFile("data/mushroom.dat")

# Set the Configurations

In [2]:
# configurations
SUPPORT = int(0.2*data.count())
print(SUPPORT)

1624


# Compute L1

In [3]:
# compute frequent-1 itemset

# Create the rdd which stores all the transactions
transactions = data.map(lambda line: line.strip().split())

# Merge all the transactions together
items=transactions.flatMap(lambda x:x)

# count the frequency of all the items in the merged rdd list
item_counts=items.map(lambda x:(x, 1)).reduceByKey(lambda x,y:x+y)

# get the frequent-1 itemset
L1=item_counts.filter(lambda x:x[1]>=SUPPORT).map(lambda x:x[0])
print(L1.count())

43


# Optimize by eliminating infrequent items in the itemsets

In [4]:
transactions.flatMap(lambda x:x).count()

186852

In [5]:
freq=sc.broadcast(set(L1.collect()))

In [6]:
# eliminate all the infrequent items
def purge_itemset(itemsets):
    # Only keep frequent items in 
    return [item for item in itemsets if item in freq.value]
transactions=transactions.map(purge_itemset)
transactions.flatMap(lambda x:x).count()

156514

# Three join_and_pruning methods(generate $C_{k+1}$ from $L_{k}$ ) are implemented. Plz only use one of them

In [20]:
# Small data algorithm, without pruning
def join_and_pruning(L_k):
    # Determine k from the first itemset if L_k is not empty
    if not L_k:
        return []

    k = len(L_k[0])

    # Generate candidate (k+1)-itemsets by joining k-itemsets
    candidate_set = set()
    n = len(L_k)
    for i in range(n):
        for j in range(i + 1, n):
            set_i = set(L_k[i])
            set_j = set(L_k[j])
            union_set = set_i | set_j
            if len(union_set) == k + 1:
                # Store as a frozen set to allow hashing and later convert to list
                candidate_set.add(frozenset(union_set))

    # Convert frozen sets back to lists
    candidates = [list(item) for item in candidate_set]

    return candidates

# Example usage
L_k = [['108', '200'], ['108', '300'], ['200', '300'], ['200', '400']]
result = join_and_pruning(L_k)
print(result)


[['200', '400', '300'], ['108', '200', '400'], ['108', '200', '300']]


In [22]:
# Small data algorithm, with pruning

def join_and_pruning(L_k):
    # Determine k from the first itemset if L_k is not empty
    if not L_k:
        return []

    k = len(L_k[0])
    threshold = k * (k + 1) // 2  # Calculate the threshold for pruning

    # Generate candidate (k+1)-itemsets by joining k-itemsets
    candidate_dict = {}
    n = len(L_k)
    for i in range(n):
        for j in range(i + 1, n):
            set_i = set(L_k[i])
            set_j = set(L_k[j])
            union_set = set_i | set_j
            if len(union_set) == k + 1:
                union_frozenset = frozenset(union_set)
                if union_frozenset in candidate_dict:
                    candidate_dict[union_frozenset] += 1
                else:
                    candidate_dict[union_frozenset] = 1

    # Filter the candidates based on the threshold
    filtered_candidates = [list(key) for key, count in candidate_dict.items() if count >= threshold]
    return filtered_candidates
L_k = [['108', '200'], ['108', '300'], ['200', '300'], ['200', '400']]
result = join_and_pruning(L_k)
print(result)

[['108', '200', '300']]


In [23]:
# Big data algorithm, with pruning
def join_and_pruning(L_k):
    if not L_k:
        return sc.parallelize([])  # Return an empty RDD if L_k is empty

    k = len(L_k[0])
    threshold = k * (k + 1) // 2  # Calculate the threshold for pruning

    # Parallelize the list and use flatMap to generate (k+1)-itemsets
    rdd = sc.parallelize(L_k)
    candidate_rdd = rdd.flatMap(lambda x: [
        (frozenset(x).union(frozenset(y)), 1) 
        for y in L_k if len(frozenset(x).union(frozenset(y))) == k + 1
    ])

    # Reduce by key to count occurrences of each (k+1)-itemset
    candidate_count_rdd = candidate_rdd.reduceByKey(lambda a, b: a + b)

    # Filter candidates based on the threshold
    filtered_candidates_rdd = candidate_count_rdd.filter(lambda x: x[1] >= threshold)

    # Convert each frozenset back to list
    final_candidates_rdd = filtered_candidates_rdd.map(lambda x: list(x[0]))

    return final_candidates_rdd.collect()

# Example usage
L_k = [['108', '200'], ['108', '300'], ['200', '300'], ['200', '400']]
rdd_result = join_and_pruning(L_k)
print(rdd_result)


[['108', '200', '300']]


# Generate $L_{k}$ from $C_{k}$

In [10]:
itemset_broadcast=sc.broadcast(transactions.map(lambda x: set(x)).collect())
def support_count(itemset):
    # Count how many transactions contain the itemset
    count = sum(1 for transaction in itemset_broadcast.value if set(itemset).issubset(transaction))
    return (itemset, count)
def get_frequent_set(C_k):
    C_k_rdd=sc.parallelize(C_k)
    # Map and filter step
    L_k = C_k_rdd \
        .map(support_count) \
        .filter(lambda x: x[1] >= SUPPORT) \
        .map(lambda x: x[0]) \
        .collect()
    return L_k

# Finally do the job

In [27]:
import time

L_k = [[item] for item in L1.collect()]
k = 2

while L_k:
    print(f'k={k}')
    
    start_time = time.time()  # Start timing for join_and_pruning
    C_k = join_and_pruning(L_k)
    end_time = time.time()  # End timing for join_and_pruning
    print(f'length of C_{k}={len(C_k)}, elapsed time={end_time - start_time:.2f} seconds')
    
    start_time = time.time()  # Start timing for get_frequent_set
    L_k = get_frequent_set(C_k)
    end_time = time.time()  # End timing for get_frequent_set
    print(f'length of L_{k}={len(L_k)}, elapsed time={end_time - start_time:.2f} seconds')
    
    k += 1


k=2
length of C_2=903, elapsed time=0.24 seconds
length of L_2=376, elapsed time=0.41 seconds
k=3
length of C_3=1764, elapsed time=0.22 seconds
length of L_3=1472, elapsed time=0.53 seconds
k=4
length of C_4=4702, elapsed time=0.32 seconds


length of L_4=3565, elapsed time=1.05 seconds
k=5
length of C_5=6579, elapsed time=0.48 seconds


length of L_5=6289, elapsed time=1.45 seconds
k=6


length of C_6=8912, elapsed time=1.43 seconds


length of L_6=8832, elapsed time=1.94 seconds
k=7


...

length of C_9=7010, elapsed time=3.71 seconds


length of L_9=7010, elapsed time=1.75 seconds
k=10


length of C_10=4004, elapsed time=2.07 seconds


length of L_10=4004, elapsed time=1.17 seconds
k=11


length of C_11=1729, elapsed time=0.87 seconds
length of L_11=1729, elapsed time=0.67 seconds
k=12
length of C_12=546, elapsed time=0.32 seconds
length of L_12=546, elapsed time=0.41 seconds
k=13
length of C_13=119, elapsed time=0.23 seconds
length of L_13=119, elapsed time=0.31 seconds
k=14
length of C_14=16, elapsed time=0.21 seconds
length of L_14=16, elapsed time=0.18 seconds
k=15
length of C_15=1, elapsed time=0.18 seconds
length of L_15=1, elapsed time=0.11 seconds
k=16
length of C_16=0, elapsed time=0.18 seconds
length of L_16=0, elapsed time=0.09 seconds
